In [140]:
import numpy as np
import sympy as sp
import pandas as pd

In [166]:
SIZE = 4
epsilon = 1e-13
start = sp.zeros(SIZE, 1)

In [142]:
def simple_iteration_method(A, B, x0):
    Allowed_A = A.col_insert(SIZE, -B)
    for i in range(SIZE):
        Allowed_A[i, i] = 0
        Allowed_A[i, :] /= -A[i, i]
    x_last = Allowed_A[:, :SIZE] * x0 + Allowed_A[:, SIZE]
    difference = np.full(1, np.nan)
    df_arr = np.array(x_last.T, dtype=np.float64)
    while True:
        x_new = Allowed_A[:, :SIZE] * x_last + Allowed_A[:, SIZE]
        df_arr = np.vstack([df_arr, np.array(x_new.T, dtype=np.float64)])
        maximum = max(abs(x_new - x_last))
        difference = np.vstack([difference, maximum.n()])
        if (maximum < epsilon):
            break
        else:
            x_last = x_new
    df = pd.DataFrame(np.append(df_arr, difference, axis = 1))
    df.columns = [f'X_{i}' if i < SIZE else 'max(|X_(k+1) - X_k|)'for i in range(SIZE+1)]
    return x_last.n(), df


def relaxation_method(A, B, x0):
    Allowed2_A = A.col_insert(SIZE, -B)
    for i in range(SIZE):
        Allowed2_A[i, :] /= -A[i, i]
    delta = Allowed2_A[:, :SIZE] * x0 + Allowed2_A[:, SIZE]
    x_new = x0
    k = []
    difference = []
    array_delta = [list(delta)]
    while True:
        max_delta = [0, 0]
        for i in range(SIZE):
            max_delta = [delta[i], i] if abs(delta[i]) > abs(max_delta[0]) else max_delta

        k.append(max_delta[1] + 1)
        difference.append(max_delta[0])

        x_new[max_delta[1]] += max_delta[0]

        if (max(abs(delta)) < epsilon):
            break

        delta[max_delta[1]] = 0
        for i in range(SIZE):
            if (i != max_delta[1]):
                delta[i] = delta[i] + Allowed2_A[i, max_delta[1]] * max_delta[0]
        array_delta.append(list(delta))
    df_data = pd.DataFrame(np.array(array_delta))
    df_difference = pd.DataFrame(np.array(difference))
    df_k = pd.DataFrame(np.array(k))
    table = pd.concat([df_k, df_difference, df_data], axis = 1, ignore_index=True)
    table.columns = [f'delta_{i-2}' if (i>=2 and i < 2+SIZE) else 'k' for i in range(SIZE+2)]
    table.columns = ['невязка' if i==1 else table.columns[i] for i in range(SIZE+2)]
    return x_new, table


def successive_over_relaxation_method(A, B, omegas, x0):
    answer = []
    df = {}
    for k in range(len(omegas)):
        x_k = x0
        
        df[k] = pd.DataFrame(np.array(x_k.T))
        
        x_k_1 = sp.zeros(SIZE, 1)
        while True:
            x_k_1[0] = (1 - omegas[k]) * x_k[0] + (omegas[k] / A[0, 0]) * (B[0] - sum([A[0, j] * x_k[j] for j in range(1, SIZE)]))
            for i in range(1, SIZE-1):
                x_k_1[i] = (1 - omegas[k]) * x_k[i] + (omegas[k] / A[i, i]) * (B[i] - sum([A[i, j] * x_k_1[j] for j in range(0, i)]) - sum([A[i, j] * x_k[j] for j in range(i+1, SIZE)]))
            x_k_1[SIZE-1] = (1 - omegas[k]) * x_k[SIZE-1] + (omegas[k] / A[SIZE-1, SIZE-1]) * (B[SIZE-1] - sum([A[SIZE-1, j] * x_k_1[j] for j in range(0, SIZE-1)]))

            df[k] = pd.concat([df[k], pd.DataFrame(np.array(x_k_1.T))], axis = 0, ignore_index = True)
            if (max([abs(x_k_1[i] - x_k[i]) for i in range(SIZE)]) < epsilon):
                answer.append(sp.Matrix(x_k_1))
                break
            x_k = sp.Matrix(x_k_1)
    return answer, df

In [143]:
A = sp.Matrix([
    [10.9, 1.2, 2.1, 0.9],
    [1.2, 11.2, 1.5, 2.5],
    [2.1, 1.5, 9.8, 1.3],
    [0.9, 2.5, 1.3, 12.1]
])
B = sp.Matrix([-7.0, 5.3, 10.3, 24.6])

In [144]:
X1, table1 = simple_iteration_method(A, B, start)
X1

Matrix([
[  -0.999999999996855],
[3.53284068665971e-12],
[    1.00000000000374],
[    2.00000000000314]])

In [145]:
D1 = A * X1 - B
D1

Matrix([
[4.92033080945475e-11],
[5.68016744750821e-11],
[ 5.2676085715575e-11],
[5.44915224054421e-11]])

In [146]:
max(abs(D1))

5.68016744750821e-11

In [147]:
table1

,X_0,X_1,X_2,X_3,max(|X_(k+1) - X_k|)
0,-0.642202,0.473214,1.05102,2.033058,NaN
1,-1.064656,-0.052548,0.846513,1.870134,0.525761878438322
2,-0.953921,0.056472,1.039125,2.032156,0.192612051656412
3,-1.01641,-0.017355,0.977217,1.980701,0.0738264756721055
4,-0.992106,0.009117,1.008733,2.007254,0.0315161082931494
5,-1.003285,-0.003635,0.995951,1.996591,0.0127820361629083
6,-0.998538,0.001655,1.001712,2.00143,0.00576173712762196
7,-1.00063,-0.000705,0.999244,1.999365,0.00246881524940945
8,-0.999724,0.00031,1.000327,2.000274,0.00108351804026063
9,-1.00012,-0.000134,0.999857,1.99988,0.000470138703607459


In [148]:
start2 = sp.zeros(SIZE, 1)
X2, table2 = relaxation_method(A, B, start2)

In [149]:
A * X2 - B

Matrix([
[ 8.88178419700125e-16],
[ -1.6864731833266e-11],
[ 2.02575733965205e-11],
[-1.01213259995347e-10]])

In [150]:
table2

,k,невязка,delta_0,delta_1,delta_2,delta_3
0,4,2.03305785123967,-0.642201834862385,0.473214285714286,1.05102040816327,2.03305785123967
1,1,-0.810068996891349,-0.810068996891349,0.0194067296340023,0.781329060549840,0
2,3,0.954915274169414,0,0.106199836443790,0.954915274169414,0.0602530658844805
3,1,-0.183974502362915,-0.183974502362915,-0.0216906020610427,0,-0.0423411371254566
4,3,0.0394231076491960,0,-0.00197904823644466,0.0394231076491960,-0.0286570832306943
5,4,-0.0328926237219303,-0.00759527762048730,-0.00725892872517626,0,-0.0328926237219303
6,1,-0.00487937290950224,-0.00487937290950224,8.31747841831742e-5,0.00436330722841932,0
7,3,0.00540888713759837,0,0.000605964738772699,0.00540888713759837,0.000362928563516695
8,1,-0.00104207917329877,-0.00104207917329877,-0.000118439788584226,0,-0.000218191542175692
9,3,0.000223302679992593,0,-6.78844858792882e-6,0.000223302679992593,-0.000140681521021238


In [167]:
omegas = [1, 0.01, 0.5, 1.5, 1.99]
start3 = sp.zeros(SIZE, 1)
ANS, tables = successive_over_relaxation_method(A, B, omegas, start3)

In [168]:
for i in range(len(omegas)):
    print(max(abs(A*ANS[i]-B)))

2.30926389122033e-14
1.11069375918760e-10
8.39328606616618e-13
2.18491891246231e-13
6.00408611717285e-13


In [169]:
tables[0]

,0,1,2,3
0,0,0,0,0
1,-0.642201834862385,0.542021625163827,1.10567259341482,1.85004572041548
2,-1.06764949951758,0.0265675185946575,1.03032175862804,1.99628491461675
3,-1.00845992097134,-0.00232526958199530,1.00266156626643,2.00082372451924
4,-1.00032479979160,-0.000505526870342285,1.00003770693805,2.00012455520404
5,-0.999961894679736,-3.69352501317881e-5,0.999980965299734,2.00000684202045
6,-0.999992831435568,2.53993031686304e-7,0.999997517387608,1.99999968105004
7,-0.999999523325748,3.52614676188015e-7,0.999999886193571,1.99999990391776
8,-1.00000000896056,3.76489217579490e-8,1.00000000890313,1.99999999193125
9,-1.00000000519389,1.16516455214314e-9,1.00000000200498,1.99999999993018


In [170]:
tables[1]

,0,1,2,3
0,0,0,0,0
1,-0.00642201834862385,0.00473902359108781,0.0105167119419852,0.0203142649024888
2,-0.0128220685888071,0.00937808491958530,0.0209079230595897,0.0404093985753263
3,-0.0191998376268148,0.0139188169490720,0.0311752818039375,0.0602879119822560
4,-0.0255550225467955,0.0183628281763622,0.0413204136996805,0.0799522856823025
...,...,...,...,...
3194,-0.999999999994770,1.30354815294748e-11,0.999999999991318,1.99999999999015
3195,-0.999999999994812,1.29331820500124e-11,0.999999999991387,1.99999999999023
3196,-0.999999999994853,1.28316858997889e-11,0.999999999991455,1.99999999999030
3197,-0.999999999994894,1.27309858385286e-11,0.999999999991523,1.99999999999038


In [171]:
tables[2]

,0,1,2,3
0,0,0,0,0
1,-0.321100917431193,0.253808977719528,0.540489717348276,0.973216187968450
2,-0.587866685727817,0.250193103901275,0.775043294379113,1.45751887806311
3,-0.763639289337967,0.188043204204169,0.883787114527385,1.70678605477569
4,-0.868870664616260,0.127503699816781,0.937533913118824,1.83870006298943
5,-0.928777337538412,0.0821215721358145,0.965549608025774,1.91006826219767
6,-0.961877728374481,0.0513624917159673,0.980724332133065,1.94934578789529
7,-0.979818097978266,0.0315442257524526,0.989145438661523,1.97124672260629
8,-0.989412743596372,0.0191408775104955,0.993880612488846,1.98358098441162
9,-0.994492666532572,0.0115176628658180,0.996557797350779,1.99058074452642


In [172]:
tables[3]

,0,1,2,3
0,0,0,0,0
1,-0.963302752293578,0.864637942332896,1.68764982647712,2.61711977439815
2,-1.43628962798529,-0.706969858058685,0.835931306017493,1.98566010608738
3,-0.615917480747488,0.329518913759561,0.885777745106068,1.88060185352275
4,-1.19866019945259,-0.0699084317176647,1.16077439125013,2.07761955466544
5,-0.945201143773198,-0.0321398601694223,0.893933228524848,1.98213040647767
6,-0.989226397865067,0.0416296379137255,1.04356830158835,1.98780969815324
7,-1.02334245159544,-0.0217343398297783,0.993134428363355,2.01654175825454
8,-0.984804279825637,0.00426570977328993,0.994277602743272,1.98963391763020
9,-1.00536469632883,0.00334970369427245,1.00587913952069,2.00379598574069


In [173]:
tables[4]

,0,1,2,3
0,0,0,0,0
1,-1.27798165137615,1.21418037352556,2.26666907205379,3.25111114792743
2,-1.68201035321375,-1.94995324992826,0.300498351057344,1.81364035419759
3,0.401197210910246,1.90090842459925,0.565190062235692,1.28848693297039
4,-2.52002731999160,-1.12587162492123,2.60940115216629,3.04820450523815
...,...,...,...,...
3377,-1.00000000000002,3.13294687050046e-14,1.00000000000007,2.00000000000003
3378,-1.00000000000001,-6.03688847561514e-14,0.999999999999949,2.00000000000001
3379,-0.999999999999955,6.13432986364499e-14,1.00000000000001,1.99999999999996
3380,-1.00000000000006,-3.43755500948227e-14,1.00000000000003,2.00000000000005
